# Train VAE model

This notebook will first try to train the current VAE model before modifying the loss function to work with count data. Specifically, the loss function uses the loss from defined in [Eraslan et al.](https://www.nature.com/articles/s41467-018-07931-2). This publication uses the zero-inflated negative binomial (ZINB) distribution, which models highly sparse and overdispersed count data. ZINB is a mixture model that is composed of
   1. A point mass at 0 to represent the excess of 0's
   2. A NB distribution to represent the count distribution

Params of ZINB conditioned on the input data are estimated. These params include the mean and dispersion parameters of the NB component (μ and θ) and the mixture coefficient that represents the weight of the point mass (π)

We adopted code from: https://github.com/theislab/dca/blob/master/dca/loss.py

More details about the new model can be found: https://docs.google.com/presentation/d/1Q_0BUbfg51OicxY4MdI0IwhdhFfJmzX0f8VyuyGNXrw/edit#slide=id.ge45eb3c133_0_56

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import matplotlib.pyplot as plt
import pandas as pd
from cm_modules import paths, utils, train_vae_modules
import scanpy as sc
import anndata

In [2]:
# Set seeds to get reproducible VAE trained models
# train_vae_modules.set_all_seeds()

In [3]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = os.path.abspath(
    os.path.join(base_dir, "test_vae_training", "config_current_vae.tsv")
)

params = utils.read_config(config_filename)

dataset_name = params["dataset_name"]

raw_compendium_filename = params["raw_compendium_filename"]
normalized_compendium_filename = params["normalized_compendium_filename"]

In [4]:
raw_compendium = pd.read_csv(raw_compendium_filename, sep="\t", index_col=0, header=0)

In [5]:
print(raw_compendium.shape)
raw_compendium.head()

(11857, 1232)


,Bacteria Actinobacteriota Actinobacteria Bifidobacteriales Bifidobacteriaceae Bifidobacterium,Bacteria Bacteroidota Bacteroidia Bacteroidales Bacteroidaceae Bacteroides,Bacteria Actinobacteriota Coriobacteriia Coriobacteriales Coriobacteriaceae Collinsella,Bacteria Firmicutes Clostridia Lachnospirales Lachnospiraceae Agathobacter,Bacteria Firmicutes Negativicutes Veillonellales-Selenomonadales Selenomonadaceae Megamonas,Bacteria Firmicutes Clostridia Lachnospirales Lachnospiraceae Blautia,Bacteria Firmicutes Clostridia Oscillospirales Ruminococcaceae Faecalibacterium,Bacteria Firmicutes Clostridia Lachnospirales Lachnospiraceae Anaerostipes,Bacteria Bacteroidota Bacteroidia Bacteroidales Prevotellaceae Prevotella,Bacteria Firmicutes Bacilli Lactobacillales Streptococcaceae Streptococcus,...,Bacteria Actinobacteriota Acidimicrobiia Microtrichales Ilumatobacteraceae NA,Bacteria Verrucomicrobiota Verrucomicrobiae Pedosphaerales Pedosphaeraceae ADurb.Bin063-1,Bacteria Proteobacteria Alphaproteobacteria Caulobacterales Caulobacteraceae PMMR1,Bacteria Bacteroidota Bacteroidia Flavobacteriales Cryomorphaceae NA,Bacteria Bacteroidota Bacteroidia Flavobacteriales Flavobacteriaceae Pseudofulvibacter,Bacteria Proteobacteria Alphaproteobacteria Rickettsiales Rickettsiaceae NA,Bacteria Bacteroidota Bacteroidia Flavobacteriales Flavobacteriaceae Gelidibacter,Bacteria Proteobacteria Gammaproteobacteria Burkholderiales Comamonadaceae Ideonella,Bacteria Proteobacteria Alphaproteobacteria Rhizobiales Xanthobacteraceae Rhodoplanes,Bacteria Proteobacteria Alphaproteobacteria Sphingomonadales Sphingomonadaceae Rhizorhapis
PRJDB5310_DRR077057,311,423,0,0,0,0,429,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PRJDB5310_DRR077058,243,313,0,0,0,13,239,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PRJDB5310_DRR077059,0,255,0,196,0,477,297,51,0,0,...,0,0,0,0,0,0,0,0,0,0
PRJDB5310_DRR077060,698,0,167,159,0,158,151,23,0,127,...,0,0,0,0,0,0,0,0,0,0
PRJDB5310_DRR077061,39,335,64,96,61,189,104,68,0,23,...,0,0,0,0,0,0,0,0,0,0


In [6]:
raw_compendium.T.to_csv(
    os.path.join(paths.LOCAL_DATA_DIR, "raw_microbiome_transposed.tsv"), sep="\t"
)

In [7]:
# TO DO:
# In the DCA paper, they log2 transformed and z-score normalized their data

# Try normalzing the data
# Here we are normalizing the microbiome count data per taxon
# so that each taxon is in the range 0-1
train_vae_modules.normalize_expression_data(
    base_dir, config_filename, raw_compendium_filename, normalized_compendium_filename
)

input: dataset contains 11857 samples and 1232 genes
Output: normalized dataset contains 11857 samples and 1232 genes


In [8]:
# test1 = pd.read_csv(raw_compendium_filename, sep="\t")
test2 = pd.read_csv(normalized_compendium_filename, sep="\t", index_col=0, header=0)

In [9]:
# test1.head()

In [10]:
test2.shape

(11857, 1232)

In [11]:
test2.head()

,Bacteria Actinobacteriota Actinobacteria Bifidobacteriales Bifidobacteriaceae Bifidobacterium,Bacteria Bacteroidota Bacteroidia Bacteroidales Bacteroidaceae Bacteroides,Bacteria Actinobacteriota Coriobacteriia Coriobacteriales Coriobacteriaceae Collinsella,Bacteria Firmicutes Clostridia Lachnospirales Lachnospiraceae Agathobacter,Bacteria Firmicutes Negativicutes Veillonellales-Selenomonadales Selenomonadaceae Megamonas,Bacteria Firmicutes Clostridia Lachnospirales Lachnospiraceae Blautia,Bacteria Firmicutes Clostridia Oscillospirales Ruminococcaceae Faecalibacterium,Bacteria Firmicutes Clostridia Lachnospirales Lachnospiraceae Anaerostipes,Bacteria Bacteroidota Bacteroidia Bacteroidales Prevotellaceae Prevotella,Bacteria Firmicutes Bacilli Lactobacillales Streptococcaceae Streptococcus,...,Bacteria Actinobacteriota Acidimicrobiia Microtrichales Ilumatobacteraceae NA,Bacteria Verrucomicrobiota Verrucomicrobiae Pedosphaerales Pedosphaeraceae ADurb.Bin063-1,Bacteria Proteobacteria Alphaproteobacteria Caulobacterales Caulobacteraceae PMMR1,Bacteria Bacteroidota Bacteroidia Flavobacteriales Cryomorphaceae NA,Bacteria Bacteroidota Bacteroidia Flavobacteriales Flavobacteriaceae Pseudofulvibacter,Bacteria Proteobacteria Alphaproteobacteria Rickettsiales Rickettsiaceae NA,Bacteria Bacteroidota Bacteroidia Flavobacteriales Flavobacteriaceae Gelidibacter,Bacteria Proteobacteria Gammaproteobacteria Burkholderiales Comamonadaceae Ideonella,Bacteria Proteobacteria Alphaproteobacteria Rhizobiales Xanthobacteraceae Rhodoplanes,Bacteria Proteobacteria Alphaproteobacteria Sphingomonadales Sphingomonadaceae Rhizorhapis
PRJDB5310_DRR077057,0.001496,0.000948,0.000000,0.000000,0.000000,0.000000,0.004278,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRJDB5310_DRR077058,0.001169,0.000702,0.000000,0.000000,0.000000,0.000046,0.002383,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRJDB5310_DRR077059,0.000000,0.000572,0.000000,0.001081,0.000000,0.001694,0.002962,0.001032,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRJDB5310_DRR077060,0.003358,0.000000,0.004034,0.000877,0.000000,0.000561,0.001506,0.000465,0.0,0.000620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRJDB5310_DRR077061,0.000188,0.000751,0.001546,0.000529,0.003158,0.000671,0.001037,0.001376,0.0,0.000112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Convert input to anndata object
test2_anndata = anndata.AnnData(test2)

In [13]:
# Save
raw_compendium_anndata_filename = os.path.join(
    paths.LOCAL_DATA_DIR, "raw_microbiome_transposed_anndata.h5ad"
)
test2_anndata.write(raw_compendium_anndata_filename)

In [14]:
# Create VAE directories if needed
output_dirs = [
    os.path.join(base_dir, dataset_name, "models"),
    os.path.join(base_dir, dataset_name, "logs"),
]

NN_architecture = params["NN_architecture"]

# Check if NN architecture directory exist otherwise create
for each_dir in output_dirs:
    sub_dir = os.path.join(each_dir, NN_architecture)
    os.makedirs(sub_dir, exist_ok=True)

In [25]:
# Train VAE on new compendium data
train_vae_modules.train_vae(config_filename, raw_compendium_anndata_filename)

here
dca: Successfully preprocessed 1232 genes and 11857 cells.
Successfully read in data
Normalized input data
input dataset contains 2625 rows and 1232 columns
input layer KerasTensor(type_spec=TensorSpec(shape=(None, 1232), dtype=tf.float32, name='count'), name='count', description="created by layer 'count'")
hidden layer KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='size_factors'), name='size_factors', description="created by layer 'size_factors'")
output KerasTensor(type_spec=TensorSpec(shape=(None, 1232), dtype=tf.float32, name=None), name='Placeholder:0', description="created by layer 'slice'")
built network
model <keras.engine.functional.Functional object at 0x7f80d43d2110>
loss <bound method ZINB.loss of <cm_modules.loss.ZINB object at 0x7f80d4693710>>
LR defined 0.001
about to compiile
Model: "model_40"
__________________________________________________________________________________________________
Layer (type)                    Output Shape    

/home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


ytrue Tensor("IteratorGetNext:2", shape=(None, 1232), dtype=float32)
ypred Tensor("model_40/lambda_1/mul:0", shape=(None, 1232), dtype=float32)
KerasTensor(type_spec=TensorSpec(shape=(None, 1232), dtype=tf.float32, name=None), name='tf.math.log_22/Log:0', description="created by layer 'tf.math.log_22'")
ytrue Tensor("IteratorGetNext:2", shape=(None, 1232), dtype=float32)
ytrue after scope Tensor("IteratorGetNext:2", shape=(None, 1232), dtype=float32)
ypred after scope Tensor("zinb_loss/mul:0", shape=(None, 1232), dtype=float32)


TypeError: in user code:

    /home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/keras/engine/training.py:830 train_function  *
        return step_function(self, iterator)
    /home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/keras/engine/training.py:813 run_step  *
        outputs = model.train_step(data)
    /home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/keras/engine/training.py:771 train_step  *
        loss = self.compiled_loss(
    /home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/keras/engine/compile_utils.py:239 __call__  *
        self._loss_metric.update_state(
    /home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/keras/metrics.py:169 decorated  *
        update_op = update_state_fn(*args, **kwargs)
    /home/alexandra/anaconda3/envs/common_microbe/lib/python3.7/site-packages/keras/metrics.py:155 update_state_fn  *
        return ag_update_state(*args, **k

In [ ]:
# Plot training and validation loss separately
# stat_logs_filename = "logs/DCA/tybalt_2layer_30latent_stats.tsv"

# stats = pd.read_csv(stat_logs_filename, sep="\t", index_col=None, header=0)

In [ ]:
# plt.plot(stats["loss"])

In [ ]:
# plt.plot(stats["val_loss"], color="orange")